In [1]:
import logging

logging.basicConfig(level=logging.WARNING)
root_logger = logging.getLogger()
root_logger.propagate = True


# Set constants

In [2]:
ema_list = [7,14]

In [3]:
test_data_path = "./tmp/test.csv"

# Load test data

In [4]:
import yfinance as yf
import pandas as pd
from NNTrade.common.candle_col_name import INDEX, OPEN,CLOSE, HIGH, LOW, VOLUME

In [5]:
import os

if not os.path.exists(test_data_path):
    data_df:pd.DataFrame =  yf.Ticker("EURUSD=X").history(start="2020-01-01",end="2024-01-01", interval="1d")
    data_df.index = data_df.index.date
    data_df.index.name = INDEX
    data_df["High"] = data_df[["Open","High","Low","Close"]].max(axis=1)
    data_df["Low"] = data_df[["Open","High","Low","Close"]].min(axis=1)
    data_df["Volume"] = 10000000
    data_df.rename({"Open":OPEN, "High": HIGH, "Low":LOW, "Close":CLOSE, "Volume":VOLUME},axis=1, inplace=True)
    data_df.to_csv(test_data_path)

In [6]:
data_df:pd.DataFrame = pd.read_csv(test_data_path, index_col=INDEX)
data_df.index = pd.to_datetime(data_df.index)
from NNTrade.indicators.ma import EmaBuilder
for ema_p in ema_list:
    ema_b = EmaBuilder(ema_p)
    data_df[ema_b.get_name(CLOSE)] = ema_b.get_for(data_df[CLOSE])
data_df

,open,high,low,close,volume,Dividends,Stock Splits,EMA(7)[close],EMA(14)[close]
start_date_time,,,,,,,,,
2020-01-01,1.122083,1.122838,1.115947,1.122083,10000000,0.0,0.0,1.122083,1.122083
2020-01-02,1.121894,1.122712,1.116682,1.122083,10000000,0.0,0.0,1.122083,1.122083
2020-01-03,1.117081,1.118068,1.112570,1.117144,10000000,0.0,0.0,1.120848,1.121424
2020-01-06,1.116246,1.120825,1.115810,1.116196,10000000,0.0,0.0,1.119685,1.120727
2020-01-07,1.119583,1.119946,1.113487,1.119799,10000000,0.0,0.0,1.119714,1.120603
...,...,...,...,...,...,...,...,...,...
2023-12-25,1.102657,1.104240,1.099989,1.102657,10000000,0.0,0.0,1.096208,1.092250
2023-12-26,1.102026,1.103997,1.100958,1.102026,10000000,0.0,0.0,1.097662,1.093554
2023-12-27,1.104301,1.112248,1.102925,1.104301,10000000,0.0,0.0,1.099322,1.094987


# Set up data source

In [7]:
data_source = {"main":data_df}

# Set strategy

In [8]:
from src.simulation.config.strategy_config import StrategyConfig
from src.simulation.realization.pyalgotrade.strategy_wrapper import StrategyWrapper, CustomValuesWrapper
from pyalgotrade.bar import BasicBar
from pyalgotrade.strategy.position import LongPosition, Position, ShortPosition
from src.simulation.models import Deal

class MyStrategy(StrategyWrapper):
    def __init__(self, barFeed,  strategy_config: StrategyConfig):
        super().__init__(barFeed, strategy_config)
        self.__last_deal:Deal = None

        self.__instrument = "main"
        
        self.__signal = float(strategy_config["delta"])
        self.__ema_p = int(strategy_config["ema_p"])
        self.__ema_name = EmaBuilder(self.__ema_p ).get_name(CLOSE)
        
        self.__ema:CustomValuesWrapper = self._get_custom_value_feed(self.__instrument, self.__ema_name)
    
    def _get_cur_equity(self)->float:
        return self.getBroker().getEquity()

    def _onBars(self, bars):

        bar: BasicBar = bars[self.__instrument]
        
        if self.__last_deal is not None:
            self.__last_deal.set_last_price(bar.getPrice())

        open_positions = [pos for pos in self.open_positions if isinstance(pos, Position) and not pos.exitActive()]
               
        if bar.getPrice()/self.__ema[-1] - 1 > self.__signal:
            if len(open_positions)==0:
                self.enterLong(self.__instrument, 1000, True)
            else:
                opened_short_pos = [pos for pos in self.open_positions if isinstance(pos, ShortPosition) and not pos.exitActive()]
                if len(opened_short_pos)>0: 
                    pos = opened_short_pos[0]
                    pos.exitMarket()
                elif len(open_positions) < 2:
                    self.enterLong(self.__instrument, 1000, True)
        elif self.__ema[-1]/bar.getPrice() - 1 > self.__signal:
            if len(open_positions)==0:
                self.enterShort(self.__instrument, 1000, True)
            else:
                opened_long_pos = [pos for pos in self.open_positions if isinstance(pos, LongPosition) and not pos.exitActive()]
                if len(opened_long_pos)>0: 
                    pos = opened_long_pos[0]
                    pos.exitMarket()
                elif len(open_positions) < 2:
                    self.enterShort(self.__instrument, 1000, True)


# Optimizing

In [9]:
from src.optimization_analyzer.optimization_analyzer import OptimizationAnalyzer, OptimizationConfig
from NNTrade.common import TimeFrame

In [10]:
from datetime import date
from src.simulation.config import StrategyId
from src.common.candle_config import CandleConfig
from src.common.candle_data_set_config import CandleDataSetConfig
from src.common.date_period import DatePeriod
from src.optimization.config.strategy_config_sets import StrategyConfigSet

str_id = StrategyId("some name", "v1")

cds_cfg = CandleDataSetConfig.BuildFrom(CandleConfig("EURUSD",TimeFrame.D),TimeFrame.D,"main")
dp_cfg = DatePeriod(date(2020,1,1), date(2024,1,1))
opt_config = OptimizationConfig(
    cds_cfg, 
    dp_cfg,
    StrategyConfigSet({
        "delta":[0,0.0001,0.0005],
        "ema_p":[7,14]}))

In [11]:
from src.simulation.realization.pyalgotrade.pyalgotrade_trading_simulator import PyalgotradeTradingSimulator,Frequency
from src.optimization_analyzer.period_splitter import DefaultPeriodSplitter,timedelta

strategy_wrapper_factory = StrategyWrapper.Factory.Single(str_id, lambda bf,cfg:MyStrategy(bf,cfg))
trading_simulator = PyalgotradeTradingSimulator(data_source, strategy_wrapper_factory)

In [12]:
from src.simulation.config.simulation_config import SimulationConfig


log = trading_simulator.get_report(str_id, SimulationConfig(cds_cfg,dp_cfg, StrategyConfig({"delta":0.0001,"ema_p":7})))
log

{'abs_cap_log': {Timestamp('2020-01-01 00:00:00'): 1000000,
                 Timestamp('2020-01-02 00:00:00'): 1000000,
                 Timestamp('2020-01-03 00:00:00'): 1000000,
                 Timestamp('2020-01-06 00:00:00'): 1000000.049829483,
                 Timestamp('2020-01-07 00:00:00'): 999996.2294101715,
                 Timestamp('2020-01-08 00:00:00'): 1000004.8806667328,
                 Timestamp('2020-01-09 00:00:00'): 1000013.1862163544,
                 Timestamp('2020-01-10 00:00:00'): 1000013.6060714722,
                 Timestamp('2020-01-13 00:00:00'): 1000012.4945640564,
                 Timestamp('2020-01-14 00:00:00'): 1000008.6817741394,
                 Timestamp('2020-01-15 00:00:00'): 1000009.6485614777,
                 Timestamp('2020-01-16 00:00:00'): 1000005.0275325775,
                 Timestamp('2020-01-17 00:00:00'): 1000008.070230484,
                 Timestamp('2020-01-20 00:00:00'): 1000012.6303434372,
                 Timestamp('2020-01-21 00:

In [13]:
from src.optimization.optimizer.realization import ComplexOptimizer
from src.optimization.strategy.realization import GridStrategyFactory
from src.optimization.report_comparer.realization import LevelYieldPerYearWithStdError

report_comparer = LevelYieldPerYearWithStdError()
grid_strategy_optimization_f = GridStrategyFactory(report_comparer)
optimizer = ComplexOptimizer(trading_simulator, grid_strategy_optimization_f)
opt_analyzer = OptimizationAnalyzer(optimizer, period_splitter=DefaultPeriodSplitter.from_days(360, 90))
analyzation_report_arr = opt_analyzer.analis_optimization_flow(str_id, opt_config,False)

100%|██████████| 12/12 [00:05<00:00,  2.35it/s]


# Report view

In [14]:
from src.simulation.models.simulation_log import SimulationLog
SimulationLog(analyzation_report_arr[0].forward.capital_log,analyzation_report_arr[0].forward.deal_list).to_dict()

{'capital_log': {Timestamp('2020-12-28 00:00:00'): 1000000,
  Timestamp('2020-12-29 00:00:00'): 1000000.3435611725,
  Timestamp('2020-12-30 00:00:00'): 1000003.2641887665,
  Timestamp('2020-12-31 00:00:00'): 1000012.6533508301,
  Timestamp('2021-01-01 00:00:00'): 999988.7278079987,
  Timestamp('2021-01-04 00:00:00'): 1000002.6485919952,
  Timestamp('2021-01-05 00:00:00'): 1000002.6035308838,
  Timestamp('2021-01-06 00:00:00'): 1000012.3386383057,
  Timestamp('2021-01-07 00:00:00'): 1000020.5054283142,
  Timestamp('2021-01-08 00:00:00'): 1000006.5727233887,
  Timestamp('2021-01-11 00:00:00'): 999989.5248413086,
  Timestamp('2021-01-12 00:00:00'): 999984.5142364502,
  Timestamp('2021-01-13 00:00:00'): 999989.3707036972,
  Timestamp('2021-01-14 00:00:00'): 999989.2966747284,
  Timestamp('2021-01-15 00:00:00'): 999990.5940294266,
  Timestamp('2021-01-18 00:00:00'): 1000007.2232484818,
  Timestamp('2021-01-19 00:00:00'): 1000005.3869485855,
  Timestamp('2021-01-20 00:00:00'): 999993.0721521

In [15]:
from src.optimization_analyzer.model import OptimizationAnalyzReport

In [16]:
analyzation_report_arr[0].to_series()

optimization_0  period   from                    2020-01-01
                         untill                  2020-04-30
                capital  yield                    -0.000013
                         yield/year               -0.000039
                         max_yield                 0.000134
                         max_fall                 -0.000153
                deals    deal_count                      18
                         success_deal_count               6
                         fail_deal_count                 10
                         avg_net_profit        -2004.435898
                         avg_net_income         9005.459837
                         avg_net_loss          -9011.260518
                         PROM                      2.401195
optimization_1  period   from                    2020-04-30
                         untill                  2020-08-28
                capital  yield                     0.000072
                         yield/year     

In [17]:
anal_rep_df = OptimizationAnalyzReport.flow_to_dataframe(analyzation_report_arr)
anal_rep_df

PROM  \
                                                                                                                  deals   
                                                                                                                forward   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forward, period, untill)             
2020-12-26                     2020-01-01                   2020-12-26              2021-03-26                 1.510894   
2021-03-26                     2020-03-31                   2021-03-26              2021-06-24                 0.618001   
2021-06-24                     2020-06-29                   2021-06-24              2021-09-22                 1.734049   
2021-09-22                     2020-09-27                   2021-09-22              2021-12-21                 1.353553   
2021-12-21                     2020-12-26                   2021-12-21              2022-03-21                 2.567273   
2022-03-21                     2021-03-26                   2022-03-21              2022-06-19                 1.731957   
2022-06-19                     2021-06-24                   2022-06-19              2022-09-17                 1.568368   
2022-09-17                     2021-09-22                   2022-09-17              2022-12-16                -2.185210   
2022-12-16                     2021-12-21                   2022-12-16              2023-03-16                 6.960593   
2023-03-16                     2022-03-21                   2023-03-16              2023-06-14                -0.773101   
2023-06-14                     2022-06-19                   2023-06-14              2023-09-12                 4.321463   
2023-09-12                     2022-09-17                   2023-09-12              2023-12-11                 4.106612   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_0   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forward, period, untill)                  
2020-12-26                     2020-01-01                   2020-12-26              2021-03-26                      2.401195   
2021-03-26                     2020-03-31                   2021-03-26              2021-06-24                     -0.021670   
2021-06-24                     2020-06-29                   2021-06-24              2021-09-22                      0.608795   
2021-09-22                     2020-09-27                   2021-09-22              2021-12-21                      0.419341   
2021-12-21                     2020-12-26                   2021-12-21              2022-03-21                      1.390379   
2022-03-21                     2021-03-26                   2022-03-21              2022-06-19                      0.618001   
2022-06-19                     2021-06-24                   2022-06-19              2022-09-17                      1.554362   
2022-09-17                     2021-09-22                   2022-09-17              2022-12-16                      1.364267   
2022-12-16                     2021-12-21                   2022-12-16              2023-03-16                      1.695877   
2023-03-16                     2022-03-21                   2023-03-16              2023-06-14                      1.888576   
2023-06-14                     2022-06-19                   2023-06-14              2023-09-12                      1.832633   
2023-09-12                     2022-09-17                   2023-09-12              2023-12-11                      0.188857   

                                                                                                 

In [18]:
anal_rep_df["yield/year"]

capital  \
                                                                                                                forward   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forward, period, untill)             
2020-12-26                     2020-01-01                   2020-12-26              2021-03-26                -0.000046   
2021-03-26                     2020-03-31                   2021-03-26              2021-06-24                 0.000199   
2021-06-24                     2020-06-29                   2021-06-24              2021-09-22                -0.000097   
2021-09-22                     2020-09-27                   2021-09-22              2021-12-21                 0.000375   
2021-12-21                     2020-12-26                   2021-12-21              2022-03-21                -0.000146   
2022-03-21                     2021-03-26                   2022-03-21              2022-06-19                -0.000177   
2022-06-19                     2021-06-24                   2022-06-19              2022-09-17                 0.000130   
2022-09-17                     2021-09-22                   2022-09-17              2022-12-16                -0.000295   
2022-12-16                     2021-12-21                   2022-12-16              2023-03-16                -0.000066   
2023-03-16                     2022-03-21                   2023-03-16              2023-06-14                 0.000096   
2023-06-14                     2022-06-19                   2023-06-14              2023-09-12                 0.000227   
2023-09-12                     2022-09-17                   2023-09-12              2023-12-11                 0.000101   

                                                                                                                              \
                                                                                                              optimization_0   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forward, period, untill)                  
2020-12-26                     2020-01-01                   2020-12-26              2021-03-26                     -0.000039   
2021-03-26                     2020-03-31                   2021-03-26              2021-06-24                      0.000152   
2021-06-24                     2020-06-29                   2021-06-24              2021-09-22                      0.000140   
2021-09-22                     2020-09-27                   2021-09-22              2021-12-21                     -0.000081   
2021-12-21                     2020-12-26                   2021-12-21              2022-03-21                      0.000029   
2022-03-21                     2021-03-26                   2022-03-21              2022-06-19                      0.000249   
2022-06-19                     2021-06-24                   2022-06-19              2022-09-17                     -0.000020   
2022-09-17                     2021-09-22                   2022-09-17              2022-12-16                      0.000189   
2022-12-16                     2021-12-21                   2022-12-16              2023-03-16                     -0.000106   
2023-03-16                     2022-03-21                   2023-03-16              2023-06-14                      0.000101   
2023-06-14                     2022-06-19                   2023-06-14              2023-09-12                     -0.000012   
2023-09-12                     2022-09-17                   2023-09-12              2023-12-11                     -0.000248   

                                                                                                                              \
                                                                                                              optimization_1   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forw

In [21]:
anal_rep_df["yield/year"].describe()

capital                                             
         forward optimization_0 optimization_1 optimization_2
count  12.000000      12.000000      12.000000      12.000000
mean    0.000025       0.000029       0.000032       0.000071
std     0.000195       0.000142       0.000168       0.000148
min    -0.000295      -0.000248      -0.000329      -0.000153
25%    -0.000110      -0.000050       0.000037      -0.000038
50%     0.000025       0.000008       0.000080       0.000066
75%     0.000147       0.000143       0.000119       0.000198
max     0.000375       0.000249       0.000220       0.000292

In [19]:
anal_rep_df[["from","untill"]]

from  \
                                                                                                                      period   
                                                                                                              optimization_0   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forward, period, untill)                  
2020-12-26                     2020-01-01                   2020-12-26              2021-03-26                    2020-01-01   
2021-03-26                     2020-03-31                   2021-03-26              2021-06-24                    2020-03-31   
2021-06-24                     2020-06-29                   2021-06-24              2021-09-22                    2020-06-29   
2021-09-22                     2020-09-27                   2021-09-22              2021-12-21                    2020-09-27   
2021-12-21                     2020-12-26                   2021-12-21              2022-03-21                    2020-12-26   
2022-03-21                     2021-03-26                   2022-03-21              2022-06-19                    2021-03-26   
2022-06-19                     2021-06-24                   2022-06-19              2022-09-17                    2021-06-24   
2022-09-17                     2021-09-22                   2022-09-17              2022-12-16                    2021-09-22   
2022-12-16                     2021-12-21                   2022-12-16              2023-03-16                    2021-12-21   
2023-03-16                     2022-03-21                   2023-03-16              2023-06-14                    2022-03-21   
2023-06-14                     2022-06-19                   2023-06-14              2023-09-12                    2022-06-19   
2023-09-12                     2022-09-17                   2023-09-12              2023-12-11                    2022-09-17   

                                                                                                                              \
                                                                                                                               
                                                                                                              optimization_1   
(optimization, period, untill) (optimization, period, from) (forward, period, from) (forward, period, untill)                  
2020-12-26                     2020-01-01                   2020-12-26              2021-03-26                    2020-04-30   
2021-03-26                     2020-03-31                   2021-03-26              2021-06-24                    2020-07-29   
2021-06-24                     2020-06-29                   2021-06-24              2021-09-22                    2020-10-27   
2021-09-22                     2020-09-27                   2021-09-22              2021-12-21                    2021-01-25   
2021-12-21                     2020-12-26                   2021-12-21              2022-03-21                    2021-04-25   
2022-03-21                     2021-03-26                   2022-03-21              2022-06-19                    2021-07-24   
2022-06-19                     2021-06-24                   2022-06-19              2022-09-17                    2021-10-22   
2022-09-17                     2021-09-22                   2022-09-17              2022-12-16                    2022-01-20   
2022-12-16                     2021-12-21                   2022-12-16              2023-03-16                    2022-04-20   
2023-03-16                     2022-03-21                   2023-03-16              2023-06-14                    2022-07-19   
2023-06-14                     2022-06-19                   2023-06-14              2023-09-12                    2022-10-17   
2023-09-12                     2022-09-17                   2023-09-12              2023-12-11                    2023-01-15   

                      

In [20]:
anal_rep_df.columns

MultiIndex([(              'PROM',   'deals',        'forward'),
            (              'PROM',   'deals', 'optimization_0'),
            (              'PROM',   'deals', 'optimization_1'),
            (              'PROM',   'deals', 'optimization_2'),
            (    'avg_net_income',   'deals',        'forward'),
            (    'avg_net_income',   'deals', 'optimization_0'),
            (    'avg_net_income',   'deals', 'optimization_1'),
            (    'avg_net_income',   'deals', 'optimization_2'),
            (      'avg_net_loss',   'deals',        'forward'),
            (      'avg_net_loss',   'deals', 'optimization_0'),
            (      'avg_net_loss',   'deals', 'optimization_1'),
            (      'avg_net_loss',   'deals', 'optimization_2'),
            (    'avg_net_profit',   'deals',        'forward'),
            (    'avg_net_profit',   'deals', 'optimization_0'),
            (    'avg_net_profit',   'deals', 'optimization_1'),
            (    'avg_net